In [18]:
import os

In [22]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry'

In [23]:
os.chdir("c:\\Users\\karakkir\\OneDrive - Tietoevry\\PREDICTING-WINE-QUALITY-USING-ML")

In [24]:
%pwd

'c:\\Users\\karakkir\\OneDrive - Tietoevry\\PREDICTING-WINE-QUALITY-USING-ML'

In [25]:
from pathlib import Path
from src.datascience.constants import *
from dataclasses import dataclass

In [26]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str



In [27]:
from src.datascience.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.param=read_yaml(PARAMS_FILE_PATH)
        self.schema=read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        param=self.param.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=param.alpha,
            l1_ratio=param.l1_ratio,
            target_column=schema.name


        )
        return model_trainer_config






In [29]:
from sklearn.linear_model import ElasticNet
import pandas as pd
from src.datascience import logger
import joblib

In [30]:
class model_train:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    
    def train_model(self):
        train=pd.read_csv(self.config.train_data_path)
        test=pd.read_csv(self.config.test_data_path)

        train_x=train.drop([self.config.target_column],axis=1)
        test_x=test.drop([self.config.target_column],axis=1)

        train_y=train[[self.config.target_column]]
        test_y=test[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)


        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

        


In [31]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_train=model_train(model_trainer_config)
    model_train.train_model()

except Exception as e:
    raise e

[2025-12-02 17:26:33,994- INFO- common - yaml file :config\config.yaml loaded successfully]
[2025-12-02 17:26:33,998- INFO- common - yaml file :params.yaml loaded successfully]
[2025-12-02 17:26:34,003- INFO- common - yaml file :schema.yaml loaded successfully]
[2025-12-02 17:26:34,007- INFO- common - created directory at:artifacts]
[2025-12-02 17:26:34,009- INFO- common - created directory at:artifacts/model_trainer]
